In [ ]:
# Divine beast bless no bug here! 
#         ┌─┐    ┌─┐
#      ┌─┘ ┴───┘ ┴──┐
#      │                   │
#      │       ───       │
#      │  ─┬┘     └┬─  │
#      │                   │
#      │       ─┴─       │
#      │                   │
#      └─┐         ┌───┘
#          │         │
#          │         │
#          │         │
#          │         └──────────────┐
#          │                                  │
#          │                                  ├─┐
#          │                                  ┌─┘
#          │                                  │
#          └─┐  ┐  ┌──────┬──┐  ┌──┘
#            │  ─┤ ─┤         │  ─┤ ─┤
#            └──┴──┘         └──┴──┘

!pip install transformers
!pip install boto3
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./
%cd ..  


     |████████████████████████████████| 2.3MB 3.9MB/s 
     |████████████████████████████████| 901kB 27.5MB/s 
     |████████████████████████████████| 3.3MB 33.9MB/s 
     |████████████████████████████████| 133kB 4.0MB/s 
     |████████████████████████████████| 7.5MB 16.6MB/s 
     |████████████████████████████████| 81kB 8.6MB/s 
     |████████████████████████████████| 153kB 43.4MB/s 
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.4 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Cloning into 'apex'...
remote: Enumerating objects: 8042, done.
remote: Counting objects: 100% (129/129), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 8042 (delta 61), reused 69 (delta 30), pack-reused 7913
Receiving obj

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
# %%writefile setup.sh
# export CUDA_HOME=/usr/local/cuda-10.1
# git clone https://github.com/NVIDIA/apex
# pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
# !sh setup.sh

In [ ]:
# data preprocessing
from sklearn.model_selection import train_test_split
import pandas as pd
import re
import numpy as np
import json
import h5py

# control
import argparse
import logging
from tqdm import trange

# pytorch
import torch
print(torch.__version__)
import torch.nn.functional as F

# transformers
from transformers import GPT2Config
from transformers import AutoTokenizer, AutoModelWithLMHead
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from transformers import BartForCausalLM, BartTokenizer,BartConfig

1.8.1+cu101


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.4) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
# the original dataframe
df_ori = pd.read_csv("./gdrive/MyDrive/COMP0087/full_dataset.csv")

In [ ]:
np.random.seed(10)
train_len = 500000
test_len = 200
subset_idx = np.random.choice(range(len(df_ori)),train_len+test_len,replace=False)
train_idx = subset_idx[0:train_len]
test_idx = subset_idx[train_len:]
df = df_ori.loc[subset_idx ,:]
# df_test = df_ori.loc[subset_idx[-100:],:]

In [ ]:
# data preprocessing
remove1 = df.loc[df.title.map(lambda x: len(x)<4 )]
remove2 = df.loc[df.ingredients.map(lambda x: len(x)<2)]
remove3 = df.loc[df.directions.map(lambda x: len(x) < 2 or len(''.join(x)) < 30)]
remove4 = df.loc[df.directions.map(lambda x: re.search('(step|mix all)', ''.join(str(x)), re.IGNORECASE)!=None)]
len(remove3)+len(remove2)+len(remove1)+len(remove4)
df.drop(df[df.title.map(lambda x: len(x)<4)].index, inplace=True)
df.drop(df[df.ingredients.map(lambda x: len(x)<2)].index, inplace=True)
df.drop(df[df.directions.map(lambda x: len(x) < 2 or len(''.join(x)) < 30)].index, inplace=True)
df.drop(df[df.directions.map(lambda x: re.search('(step|mix all)', ''.join(str(x)), re.IGNORECASE)!=None)].index, inplace=True)

In [ ]:
df.drop(df[df.title.map(lambda x: len(x)<4)].index, inplace=True)
df.drop(df[df.ingredients.map(lambda x: len(x)<2)].index, inplace=True)
df.drop(df[df.directions.map(lambda x: len(x) < 2 or len(''.join(x)) < 30)].index, inplace=True)
df.drop(df[df.directions.map(lambda x: re.search('(step|mix all)', ''.join(str(x)), re.IGNORECASE)!=None)].index, inplace=True)

In [ ]:
len(df)

470922

In [ ]:
df.reset_index(inplace=True)
train, test = train_test_split(df, test_size=0.05) #use 5% for test set
# we only want first 10000 and 100 for train/test
# this has an error: train[75400:75600]
train = train[50000:400000]
test = test[0:100]

In [ ]:
def df_to_plaintext_file(input_df, output_file):
    print("Writing to", output_file)
    with open(output_file, 'w') as f:
        for index, row in input_df.iterrows():
            if index%100000==0:
                print(index)
            if type(row.NER)!=str:
                continue
            title = row.title
            directions = json.loads(row.directions)
            ingredients = json.loads(row.ingredients)
            ner = json.loads(row.NER)
            res = "<RECIPE_START> <INPUT_START> " + " <NEXT_INPUT> ".join(ner) + " <INPUT_END> <INGR_START> " + \
              " <NEXT_INGR> ".join(ingredients) + " <INGR_END> <INSTR_START> " + \
              " <NEXT_INSTR> ".join(directions) + " <INSTR_END> <TITLE_START> " + title + " <TITLE_END> <RECIPE_END>"
            f.write("{}\n".format(res))

In [ ]:
df_to_plaintext_file(train, 'unsupervised_train.txt')
df_to_plaintext_file(test, 'unsupervised_test.txt')

Writing to unsupervised_train.txt
300000
0
100000
400000
200000
Writing to unsupervised_test.txt


In [ ]:
#from transformers import GPT2Tokenizer

In [ ]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
special_tokens = {
    "additional_special_tokens": [
        "<TITLE_START>",
        "<TITLE_END>",
        "<INSTR_START>",
        "<NEXT_INSTR>",
        "<INSTR_END>",
        "<INGR_START>",
        "<NEXT_INGR>",
        "<INGR_END>",
        "<RECIPE_START>",
        "<RECIPE_END>",
        "<INPUT_START>",
        "<INPUT_END>",
        "<NEXT_INPUT>"
    ]
}

tokenizer.add_special_tokens(special_tokens)

end_token_id = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]

hf = h5py.File("unsupervised.h5", "w")
for filename in ["test", "train"]:
    out_np = []
    data = open("unsupervised_"+filename+".txt", "r")
    num = 0
    rows = 0
    last=[]
    for line in data:
        num+=1
        if num%10000 == 0:
            print("Read "+str(num)+" Written: "+str(rows))
        # print(line)
        text_tokens = tokenizer(line)['input_ids']
        # print(text_tokens[0:5])
        if len(text_tokens) > 1024: #Recipe won't fit the model
            continue

        # text_tokens_ids = tokenizer.convert_tokens_to_ids(text_tokens)
        text_tokens_ids = text_tokens
        # print(text_tokens_ids[0:5])

        if (len(last) + len(text_tokens_ids)) <= 1024:
            last+=text_tokens_ids
        else:
            while len(last) < 1024:
                last.append(end_token_id)
            out_np.append(last)
            last=text_tokens_ids
            rows+=1
    out_mat = np.matrix(out_np)
    print(out_mat.shape)
    # print(out_mat)
    hf.create_dataset(filename, data=out_mat)
hf.close()



(27, 1024)


Token indices sequence length is longer than the specified maximum sequence length for this model (1127 > 1024). Running this sequence through the model will result in indexing errors


Read 10000 Written: 2649
Read 20000 Written: 5321
Read 30000 Written: 7959
Read 40000 Written: 10610
Read 50000 Written: 13248
Read 60000 Written: 15901
Read 70000 Written: 18562
Read 80000 Written: 21200
Read 90000 Written: 23839
Read 100000 Written: 26514
Read 110000 Written: 29148
Read 120000 Written: 31803
Read 130000 Written: 34482
Read 140000 Written: 37158
Read 150000 Written: 39790
Read 160000 Written: 42460
Read 170000 Written: 45131
Read 180000 Written: 47826
Read 190000 Written: 50448
Read 200000 Written: 53104
Read 210000 Written: 55770
Read 220000 Written: 58419
Read 230000 Written: 61078
Read 240000 Written: 63719
Read 250000 Written: 66361
Read 260000 Written: 68977
Read 270000 Written: 71636
Read 280000 Written: 74302
Read 290000 Written: 76923
Read 300000 Written: 79554
Read 310000 Written: 82232
Read 320000 Written: 84913
Read 330000 Written: 87582
Read 340000 Written: 90260
Read 350000 Written: 92917
(94095, 1024)


In [ ]:
tokenizer

PreTrainedTokenizer(name_or_path='facebook/bart-base', vocab_size=50265, model_max_len=1024, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True), 'additional_special_tokens': ['<TITLE_START>', '<TITLE_END>', '<INSTR_START>', '<NEXT_INSTR>', '<INSTR_END>', '<INGR_START>', '<NEXT_INGR>', '<INGR_END>', '<RECIPE_START>', 

In [ ]:
toke_result = tokenizer("<RECIPE_START> <INPUT_START> water <NEXT_INPUT> butter")
toke_result

{'input_ids': [0, 50273, 50275, 5412, 50277, 4297, 1334, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer.decode([50274])

'<RECIPE_END>'

In [ ]:
#raw_text = 'tomato,egg'
#prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
# tokenizer.encode(prepared_input)

# fine-tune with BART

In [ ]:
# fine-tuning model with bart
# logging info
import logging
logger = logging.getLogger(__name__)

import glob
import logging
import random
import gc
import boto3
import shutil

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler
from torch.utils.data.distributed import DistributedSampler
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm, trange
import logging
import math
import os
from dataclasses import dataclass, field
from typing import Optional
from torch.utils.data import Dataset
import h5py
import torch

from transformers import WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup
from transformers import BartForCausalLM, BartTokenizer,BartConfig,BartForConditionalGeneration

In [ ]:
# data loading
class TextDataset(Dataset):
    def __init__(self, tokenizer, file_path='train', block_size=512):
        cached_features_file = "unsupervised.h5"

        logger.info("Loading features from cached file %s", cached_features_file)
        with h5py.File(cached_features_file, 'r') as f:
            if file_path=='test':
                self.examples = f[file_path][:] #this is a dev set, 10% of a test set
            else:
                self.examples = f[file_path][:]

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item])

def load_and_cache_examples(args, tokenizer, evaluate=False):
    dataset = TextDataset(tokenizer, file_path="test" if evaluate else "train", block_size=args.block_size)
    print(dataset[0:5])
    return dataset

In [ ]:
parser = argparse.ArgumentParser()
## Required parameters
parser.add_argument("--train_data_file", default="unsupervised.h5", type=str, required=False,
                        help="The input training data file (a text file).")
parser.add_argument("--output_dir", default="./gdrive/MyDrive/COMP0087/model_checkpoint", type=str, required=False,
                        help="The output directory where the model predictions and checkpoints will be written.")

## Other parameters
parser.add_argument("--model_type", default="facebook/bart-base" ,type=str,   #"facebook/bart-base"
                        help="The model architecture to be fine-tuned.")
parser.add_argument("--model_name_or_path", default="facebook/bart-base", type=str,
                        help="The model checkpoint for weights initialization.")

parser.add_argument("--eval_data_file", default=None, type=str,
                        help="An optional input evaluation data file to evaluate the perplexity on (a text file).")

parser.add_argument("--config_name", default="", type=str,
                        help="Optional pretrained config name or path if not the same as model_name_or_path")
parser.add_argument("--tokenizer_name", default="", type=str,
                        help="Optional pretrained tokenizer name or path if not the same as model_name_or_path")
parser.add_argument("--cache_dir", default="", type=str,
                        help="Optional directory to store the pre-trained models downloaded from s3 (instread of the default one)")
parser.add_argument("--block_size", default=-1, type=int,
                        help="Optional input sequence length after tokenization."
                             "The training dataset will be truncated in block of this size for training."
                             "Default to the model max input length for single sentence inputs (take into account special tokens).")
parser.add_argument("--do_train", action='store_true',
                        help="Whether to run training.")
parser.add_argument("--do_eval", action='store_true',
                        help="Whether to run eval on the dev set.")
parser.add_argument("--evaluate_during_training", action='store_true',
                        help="Run evaluation during training at each logging step.")
parser.add_argument("--do_lower_case", action='store_true',
                        help="Set this flag if you are using an uncased model.")

parser.add_argument("--per_gpu_train_batch_size", default=3, type=int,
                        help="Batch size per GPU/CPU for training.")
parser.add_argument("--per_gpu_eval_batch_size", default=4, type=int,
                        help="Batch size per GPU/CPU for evaluation.")
parser.add_argument('--gradient_accumulation_steps', type=int, default=1,
                        help="Number of updates steps to accumulate before performing a backward/update pass.")
parser.add_argument("--learning_rate", default=5e-5, type=float,
                        help="The initial learning rate for Adam.")
parser.add_argument("--weight_decay", default=0.0, type=float,
                        help="Weight deay if we apply some.")
parser.add_argument("--adam_epsilon", default=1e-8, type=float,
                        help="Epsilon for Adam optimizer.")
parser.add_argument("--max_grad_norm", default=1.0, type=float,
                        help="Max gradient norm.")
parser.add_argument("--num_train_epochs", default=2, type=float,
                        help="Total number of training epochs to perform.")
parser.add_argument("--max_steps", default=-1, type=int,
                        help="If > 0: set total number of training steps to perform. Override num_train_epochs.")
parser.add_argument("--warmup_steps", default=0, type=int,
                        help="Linear warmup over warmup_steps.")

parser.add_argument('--logging_steps', type=int, default=50,
                        help="Log every X updates steps.")
parser.add_argument('--save_steps', type=int, default=50,
                        help="Save checkpoint every X updates steps.")
parser.add_argument("--eval_all_checkpoints", action='store_true',
                        help="Evaluate all checkpoints starting with the same prefix as model_name_or_path ending and ending with step number")
parser.add_argument("--no_cuda", action='store_true',
                        help="Avoid using CUDA when available")
parser.add_argument('--overwrite_output_dir', action='store_true',
                        help="Overwrite the content of the output directory")
parser.add_argument('--overwrite_cache', action='store_true',
                        help="Overwrite the cached training and evaluation sets")
parser.add_argument("--aws_bucket", default="", type=str,
                        help="Whether to upload to specified bucket.")
args, unknown = parser.parse_known_args()

def setup_args_for_model(args):
  args.model_type ="facebook/bart-base" # "facebook/bart-base"
  args.model_type = ""
  
  

In [ ]:
import random

def shuufle_instruction(this_batch):
  ins_element_id = tokenizer.convert_tokens_to_ids(["<NEXT_INSTR>"])[0]
  recipe_end_id = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]
  end_index = (this_batch == recipe_end_id).nonzero()[0]
  # ins_element_id = tokenizer.convert_tokens_to_ids(["<NEXT_INSTR>"])[0]
  ing_index = (this_batch == ins_element_id).nonzero()
  ing_index = ing_index[ing_index<end_index]
  if len(ing_index) > 2:
    split_result = torch.tensor_split(this_batch, ing_index.squeeze())
    split_list = list(split_result[1:-1])
    random.shuffle(split_list)
    shuffle_batch = torch.cat((split_result[0],torch.cat(split_list),split_result[-1]))
    return shuffle_batch
  else: 
    return this_batch

In [ ]:
import torch
import torch.nn as nn
# from transformer import AlbertTokenizer, AlbertForMultipleChoice
from transformers import BartModel, BartTokenizer, BartConfig, BartForCausalLM
from torch.nn import CrossEntropyLoss, MSELoss


class bartForBinaryClassifier(nn.Module):
    def __init__(self, tokenizer, token_size=1024):
        super(bartForBinaryClassifier, self).__init__()

        # device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
        self.bart = BartForCausalLM.from_pretrained("facebook/bart-base")
        self.bart.resize_token_embeddings(len(tokenizer))
        # self.lm_head = nn.Linear(self.bart.config.d_model, self.bart.config.vocab_size, bias=False)

        # add an module for consistency checking
        self.consist_head = nn.Linear(self.bart.config.d_model, 1, bias=False)
        self.consist_logit = nn.Linear(token_size-1, 1)
        self.consist_activation = nn.Sigmoid()
        # add an module for checking the reasonability of ingredients unit
        # self.units_logit = nn.Linear(token_size-1, 1)
        # self.units_activation = nn.Sigmoid()
        self.cuda()

    def forward(self, batch, ins_start_id, ing_start_id = None, labels=None):
        # print(batch.shape)
        outputs = self.bart(batch,output_hidden_states=True)
        last_hidden = outputs["hidden_states"][-1]
        # print(last_hidden.shape)
        consist_head = self.consist_head(last_hidden)
        consist_head = consist_head.squeeze(dim=2)
        ins_subvector = torch.empty(0, consist_head.size(1)).to(args.device)
        # ing_subvector = torch.empty(0, consist_head.size(1)).to(args.device)
        # print("consist_head:",consist_head.shape)
        for i in range(len(consist_head)):
            sample = consist_head[i]

            # the start index of ingredients
            # ing_index = (batch[i] == ing_start_id).nonzero()[0]
            # the start index of instruction
            
            # move the non-zero elements of ingredients vector to left
            # ing_z = torch.zeros(consist_head.size(1)).to(args.device)
            # ing_z[0:(ins_index-ing_index)] = sample[ing_index:ins_index]
            # ing_subvector = torch.cat((ing_subvector, ing_z.unsqueeze(0)))
            ins_index = (batch[i] == ins_start_id).nonzero()[0]
            sample[0:ins_index] = 0
            nz = sample.nonzero().squeeze()
            # move the non-zero elements of instructions vector to left
            ins_z = torch.zeros(sample.numel() - nz.numel()).to(args.device)
            ins_z = torch.cat((sample[nz], ins_z)).unsqueeze(0)
            ins_subvector = torch.cat((ins_subvector, ins_z))
            # print(ins_subvector.shape)

        # print(ins_subvector.shape)
        consist_output = self.consist_logit(ins_subvector.half().to(args.device))
        consist_output = self.consist_activation(consist_output)
        # print(consist_output.shape)

        # units_output = self.units_logit(ing_subvector.half().to(args.device))
        # units_output = self.units_activation(units_output)
        # print(units_output.shape)

        # the usual prediction output
        logits = outputs["logits"]
        # output_token = logits.argmax(dim=2)



        # output = self.out(output_token.float())

        # we apply dropout to the sequence output, tensor has shape (batch_size, sequence_length, 768)
        # sequence_output = self.dropout(sequence_output)

        # next, we apply the linear layer. The linear layer (which applies a linear transformation)
        # takes as input the hidden states of all tokens (so seq_len times a vector of size 768, each corresponding to
        # a single token in the input sequence) and outputs 2 numbers (scores, or logits) for every token
        # so the logits are of shape (batch_size, sequence_length, 2)
        # logits = self.dropout(sequence_output)

        return logits, consist_output

In [ ]:
def save_model(args,model,tokenizer,model_class,tokenizer_class):
  # Create output directory if neede
  if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

  logger.info("Saving model checkpoint to %s", args.output_dir)
  # Save a trained model, configuration and tokenizer using `save_pretrained()`.
  # They can then be reloaded using `from_pretrained()`
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  model_to_save.save_pretrained(args.output_dir)
  tokenizer.save_pretrained(args.output_dir)

  # Good practice: save your training arguments together with the trained model
  torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))

  # Load a trained model and vocabulary that you have fine-tuned
  model = model_class.from_pretrained(args.output_dir)
  tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
  model.to(args.device)


In [ ]:
from torch.nn import CrossEntropyLoss, MSELoss


def train_seq(args, train_dataset, model, tokenizer):
    """ Train the model """

    args.train_batch_size = args.per_gpu_train_batch_size * max(1, args.n_gpu)
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)

    if args.max_steps > 0:
        t_total = args.max_steps
        args.num_train_epochs = args.max_steps // (len(train_dataloader) // args.gradient_accumulation_steps) + 1
    else:
        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps,
                                                num_training_steps=t_total)

    try:
        from apex import amp
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")

    model, optimizer = amp.initialize(model, optimizer, opt_level="O2")

    # Train!
    global_step = 0
    epoch = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()

    train_iterator = trange(int(args.num_train_epochs), desc="Epoch", disable=True)
    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=True)
        

        if epoch == 1:
          # print("start discriminative part")
          # second step: fix the discriminative part 
          fix_layers = [mybart.consist_head, mybart.consist_logit, mybart.consist_activation]
                # mybart.units_logit, mybart.units_activation]
          for layer in fix_layers:
            for parameter in layer.parameters():
              parameter.requires_grad = False


        epoch +=1
        for step, batch in enumerate(epoch_iterator):

            batch_drop = 0
            ins_start_id = tokenizer.convert_tokens_to_ids(["<INSTR_START>"])[0]
            ing_start_id = tokenizer.convert_tokens_to_ids(["<INGR_START>"])[0]

            random_shuffle = torch.randint(2, (batch.size()[0], 1),device = args.device)
            # print(random_shuffle)
            for batch_no in range(len(batch)):
              # print((batch[batch_no] == ins_start_id).nonzero())
              if (batch[batch_no] == ins_start_id).nonzero().size()[0] == 0:
                print("An error happens, break")
                batch_drop = 1
                break

              if random_shuffle[batch_no] == 1:
                # print(batch[batch_no])
                batch[batch_no] = shuufle_instruction(batch[batch_no])
              
            if batch_drop == 1:
              print("An error in this batch, break")
              torch.cuda.empty_cache()
              continue

            # print(batch.shape)
            inputs, labels = (batch[:, 0:-1], batch[:, 1:])
            inputs = inputs.to(args.device)
            labels = labels.to(args.device)
            # label_seq = torch.zeros(batch.size()[0],1).to(args.device)

            model.train()

            loss_CE = CrossEntropyLoss()
            loss_MSE = MSELoss()

            # output,consist_output = mybart(batch[:,0:-1], ins_start_id)
            
            
            logits, consist_output = model(inputs, ins_start_id, ing_start_id,)
            # outputs = model(inputs, labels=labels)
            # logits = outputs['logits']

            # loss adjustment
            bart_vocab_size = 50278
            loss1 = loss_CE(logits.reshape(-1, bart_vocab_size), labels.reshape(-1))
            loss2 = loss_MSE(consist_output, random_shuffle.float())
            loss = loss1 + loss2
            # print("loss comp: ", loss, outputs['loss'])
            if global_step % 50 == 0:
              logger.info(step)
              print("step", global_step)
              print("loss:",loss.item())

            if global_step % 10000 == 0 and global_step !=0:
              model_checkpoint = mybart.bart
              model_class = BartForCausalLM
              tokenizer_class = BartTokenizer
              save_model(args,model_checkpoint,tokenizer,model_class,tokenizer_class)

            if args.gradient_accumulation_steps > 1:
                loss = loss / args.gradient_accumulation_steps

            with amp.scale_loss(loss, optimizer) as scaled_loss:
                # print(scaled_loss)
                scaled_loss.backward()

            tr_loss += loss.item()
            if (step) % args.gradient_accumulation_steps == 0:
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                # if args.logging_steps > 0 and global_step % args.logging_steps == 0:
                    # Log metrics
                    # tb_writer.add_scalar('lr', scheduler.get_lr()[0], global_step)
                    # tb_writer.add_scalar('loss', (tr_loss - logging_loss) / args.logging_steps, global_step)
                    # logging_loss = tr_loss

            if args.max_steps > 0 and global_step > 2:
                epoch_iterator.close()
                break
            del inputs, labels,logits,consist_output , loss1,loss2, loss
            torch.cuda.empty_cache()
            # break

    return global_step, tr_loss / global_step, batch

In [ ]:
def model_init(args,logger, model_class, tokenizer):
  if args.eval_data_file is None and args.do_eval:
    raise ValueError("Cannot do evaluation without an evaluation data file. Either supply a file to --eval_data_file "
                         "or remove the --do_eval argument.")
  if os.path.exists(args.output_dir) and os.listdir(args.output_dir) and args.do_train and not args.overwrite_output_dir:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args.output_dir))
  device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
  args.n_gpu = torch.cuda.device_count()
  args.device = device

  # Setup logging
  logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                        datefmt = '%m/%d/%Y %H:%M:%S',
                        level = logging.INFO)

  model = bartForBinaryClassifier(tokenizer)

  if args.block_size <= 0:
    args.block_size = tokenizer.max_len_single_sentence  # Our input block size will be the max possible for the model
  args.block_size = min(args.block_size, tokenizer.max_len_single_sentence)
  model.to(args.device)

  logger.info("Training/evaluation parameters %s", args)
  return model, logger

In [ ]:
torch.manual_seed(0)
train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)
mybart, logger = model_init(args,logger,BartForCausalLM,tokenizer)
global_step, tr_loss, batch = train_seq(args, train_dataset, mybart, tokenizer)

tensor([[    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274]])


05/21/2021 09:42:05 - INFO - filelock -   Lock 140383132559824 acquired on /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.3c4e938c08a46506ff9a4be12e0f858cd714b9a5c76090d571e7a4aa95cae853.lock


05/21/2021 09:42:05 - INFO - filelock -   Lock 140383132559824 released on /root/.cache/huggingface/transformers/f5310d276a6d1648d00c32fadc8bf7b4607e0fbd5b404fc4a0045960aa2bdfdb.3c4e938c08a46506ff9a4be12e0f858cd714b9a5c76090d571e7a4aa95cae853.lock
05/21/2021 09:42:05 - INFO - filelock -   Lock 140383131961168 acquired on /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.9faea28a6782a9589c09b1942c039943df02232d83d2ac288a69ddfa928eae22.lock


05/21/2021 09:42:20 - INFO - filelock -   Lock 140383131961168 released on /root/.cache/huggingface/transformers/486355ec722ef05fd480e999d4c763be56549ae930f6a3742ee721a5d2a05647.9faea28a6782a9589c09b1942c039943df02232d83d2ac288a69ddfa928eae22.lock


Some weights of the model checkpoint at facebook/bart-base were not used when initializing BartForCausalLM: ['model.encoder.layers.2.self_attn.k_proj.bias', 'model.encoder.layers.4.fc1.weight', 'model.encoder.layers.0.self_attn.v_proj.bias', 'model.encoder.layers.4.self_attn.out_proj.weight', 'model.encoder.layers.1.self_attn.v_proj.bias', 'model.encoder.layers.5.self_attn.k_proj.bias', 'model.encoder.layers.1.self_attn.v_proj.weight', 'model.encoder.layers.1.self_attn.k_proj.weight', 'model.encoder.layers.5.self_attn.out_proj.bias', 'model.encoder.layers.2.self_attn_layer_norm.bias', 'model.encoder.layers.3.self_attn.q_proj.weight', 'model.encoder.layers.2.self_attn_layer_norm.weight', 'model.encoder.layers.0.self_attn.q_proj.weight', 'model.encoder.layers.2.final_layer_norm.weight', 'model.encoder.layers.0.self_attn_layer_norm.bias', 'model.encoder.layers.5.self_attn.q_proj.weight', 'model.encoder.layers.1.fc2.weight', 'model.encoder.layers.3.self_attn_layer_norm.weight', 'model.enco

...

step 62600
loss: 1.9403263330459595


05/21/2021 16:04:46 - INFO - __main__ -   31295


step 62650
loss: 1.2383302450180054


05/21/2021 16:05:04 - INFO - __main__ -   31345


step 62700
loss: 1.615312933921814


## evaluation

In [ ]:
def save_model(args,model,tokenizer,model_class,tokenizer_class):
  # Create output directory if neede
  if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

  logger.info("Saving model checkpoint to %s", args.output_dir)
  # Save a trained model, configuration and tokenizer using `save_pretrained()`.
  # They can then be reloaded using `from_pretrained()`
  model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
  model_to_save.save_pretrained(args.output_dir)
  tokenizer.save_pretrained(args.output_dir)

  # Good practice: save your training arguments together with the trained model
  torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))

  # Load a trained model and vocabulary that you have fine-tuned
  model = model_class.from_pretrained(args.output_dir)
  tokenizer = tokenizer_class.from_pretrained(args.output_dir, do_lower_case=args.do_lower_case)
  model.to(args.device)



In [ ]:
model = mybart.bart
model_class = BartForCausalLM
tokenizer_class = BartTokenizer
save_model(args,model,tokenizer,model_class,tokenizer_class)

05/21/2021 16:05:12 - INFO - __main__ -   Saving model checkpoint to ./gdrive/MyDrive/COMP0087/model_checkpoint


In [ ]:
# model = BartForCausalLM.from_pretrained(args.output_dir)

In [ ]:
def evaluate(args, model, tokenizer, prefix=""):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args.output_dir

    eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True)

    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)

    args.eval_batch_size = args.per_gpu_eval_batch_size * max(1, args.n_gpu)
    # Note that DistributedSampler samples randomly
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args.eval_batch_size)
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        batch = batch.to(args.device)

        with torch.no_grad():
            outputs = model(batch, labels=batch)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {
        "perplexity": perplexity
    }

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [ ]:
# Evaluation
results = {}
checkpoints = [args.output_dir]
if args.eval_all_checkpoints:
  checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args.output_dir + '/**/' + WEIGHTS_NAME, recursive=True)))
  logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
logger.info("Evaluate the following checkpoints: %s", checkpoints)
for checkpoint in checkpoints:
  global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
  model_class = BartForCausalLM#BartForConditionalGeneration#GPT2LMHeadModel # BartForCausalLM
  model = model_class.from_pretrained(checkpoint)
  model.to(args.device)
  result = evaluate(args, model, tokenizer, prefix=global_step)
  result = dict((k + '_{}'.format(global_step), v) for k, v in result.items())
  results.update(result)

05/21/2021 16:05:17 - INFO - __main__ -   Evaluate the following checkpoints: ['./gdrive/MyDrive/COMP0087/model_checkpoint']
05/21/2021 16:05:19 - INFO - __main__ -   Loading features from cached file unsupervised.h5
05/21/2021 16:05:19 - INFO - __main__ -   ***** Running evaluation  *****
05/21/2021 16:05:19 - INFO - __main__ -     Num examples = 27
05/21/2021 16:05:19 - INFO - __main__ -     Batch size = 4
Evaluating:  14%|█▍        | 1/7 [00:00<00:00,  6.65it/s]

tensor([[    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274],
        [    0, 50273, 50275,  ..., 50274, 50274, 50274]])


Evaluating: 100%|██████████| 7/7 [00:00<00:00,  7.25it/s]
05/21/2021 16:05:20 - INFO - __main__ -   ***** Eval results  *****
05/21/2021 16:05:20 - INFO - __main__ -     perplexity = tensor(11198.6689)


# generation

In [ ]:
#model loading
#tokenizer = AutoTokenizer.from_pretrained("mbien/recipenlg")
#model = AutoModelWithLMHead.from_pretrained("mbien/recipenlg")

In [ ]:
raw_text = 'tomato,egg'
prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
tokenizer.encode(prepared_input)[0:-1]

[0, 50273, 50275, 33063, 3938, 50277, 38299]

In [ ]:
#new_model = BartForCausalLM.from_pretrained(args.output_dir)
#new_tokenizer = BartTokenizer.from_pretrained(args.output_dir)
#new_model = GPT2LMHeadModel.from_pretrained(args.output_dir)
#new_tokenizer = GPT2LMHeadModel.from_pretrained(args.output_dir)
#new_model = BartForConditionalGeneration.from_pretrained(args.output_dir)
#new_tokenizer= BartForConditionalGeneration.from_pretrained(args.output_dir)

In [ ]:
def set_seed(args):
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-float('Inf')):
    """ Filter a distribution of logits using top-k and/or nucleus (top-p) filtering
        Args:
            logits: logits distribution shape (vocabulary size)
            top_k > 0: keep only top k tokens with highest probability (top-k filtering).
            top_p > 0.0: keep the top tokens with cumulative probability >= top_p (nucleus filtering).
                Nucleus filtering is described in Holtzman et al. (http://arxiv.org/abs/1904.09751)
        From: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
    """
    assert logits.dim() == 1  # batch size 1 for now - could be updated for more but the code would be less clear
    top_k = min(top_k, logits.size(-1))  # Safety check
    if top_k > 0:
        # Remove all tokens with a probability less than the last token of the top-k
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probs > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value
    return logits


def sample_sequence(model, length, context, tokenizer, num_samples=1, temperature=1, top_k=0, top_p=0.0, device='cpu'):
    end_token = tokenizer.convert_tokens_to_ids(["<RECIPE_END>"])[0]
    context = torch.tensor(context, dtype=torch.long, device=device)
    context = context.unsqueeze(0).repeat(num_samples, 1)
    generated = context
    with torch.no_grad():
        for _ in range(length):
            inputs = {'input_ids': generated}
            outputs = model(**inputs)  # Note: we could also use 'past' with GPT-2/Transfo-XL/XLNet (cached hidden-states)
            # print(outputs)
            next_token_logits = outputs["logits"][0, -1, :] / temperature
            
            filtered_logits = top_k_top_p_filtering(next_token_logits, top_k=top_k, top_p=top_p)
            next_token = torch.multinomial(F.softmax(filtered_logits, dim=-1), num_samples=1)
            generated = torch.cat((generated, next_token.unsqueeze(0)), dim=1)
            #print(next_token)
            if next_token.item() == end_token:
                break
    return generated

In [ ]:
# raw_text = args.prompt if args.prompt else input("Comma-separated ingredients, semicolon to close the list >>> ")
raw_text = 'tomato, potato, beef'

prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
context_tokens = tokenizer.encode(prepared_input)[0:-1]
out = sample_sequence(
            model=mybart.bart,
            context=context_tokens,
            tokenizer=tokenizer,
            length=800,
            device = args.device
        )
out = out[0, len(context_tokens):].tolist()
text = tokenizer.decode(out, clean_up_tokenization_spaces=True)
if "<RECIPE_END>" not in text:
  print(text)
  print("Failed to generate, recipe's too long")
  full_text = prepared_input + text
  print(full_text)
else:
  full_text = prepared_input + text
  print(full_text)
  markdown = re.sub("<RECIPE_(START|END)>", "", full_text)
  recipe_n_title = markdown.split("<TITLE_START>")
  title = "# " + recipe_n_title[1].replace("<TITLE_END>", "") + " #\n"
  markdown = recipe_n_title[0].replace("<INPUT_START>", "## Input ingredients ##\n`").replace("<INPUT_END>", "`\n")
  markdown = markdown.replace("<NEXT_INPUT>", "`\n`").replace("<INGR_START>", "## Ingredients ##\n* ").replace("<NEXT_INGR>", "\n* ").replace("<INGR_END>", "\n")
  markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n1) ").replace("<NEXT_INSTR>", "\n1) ").replace("<INSTR_END>", "\n")
  markdown = re.sub("$ +#", "#", markdown)
  markdown = re.sub("( +`|` +)", "`", markdown)
  print(title+markdown)#


 <RECIPE_START> <INPUT_START> tomato <NEXT_INPUT>  potato <NEXT_INPUT>  beef<NEXT_INPUT> vegetable sprouts <NEXT_INPUT> butter <NEXT_INPUT> thyme <INPUT_END> <INGR_START> 1 medium tomato, chopped <NEXT_INGR> 1 medium potato, diced <NEXT_INGR> 8 marinated beef sprouts <NEXT_INGR> 2 tablespoons butter <NEXT_INGR> 1 teaspoon dried thyme <NEXT_INGR> 1 stick artichoke hearts, minced <INGR_END> <INSTR_START> Use very finely chopped tomatoes and celery with low cut off sauce. <NEXT_INSTR> Set aside. <NEXT_INSTR> When I'm paaing, squeeze hot grease with hand. When my machine starts to start to turn high. <NEXT_INSTR> Now, tactfully but slightly the sun, steam the tomatoes until they start to turn golden brown (about 5 minutes). Watch them carefully. <NEXT_INSTR> If your sun heats, place them on tray at the stove for 15-20 minutes. <NEXT_INSTR> When they smell like burning, lower heat again. <NEXT_INSTR> Once oil is fully shaking the tomatoes you added them in 1 tablespoon. <NEXT_INSTR> Serve w

In [ ]:
print(title+markdown)

#  Kwant Needed Finger Soup with Tomato   #
  ## Input ingredients ##
`tomato`
`potato`
`beef`
`vegetable sprouts`
`butter`
`thyme`
 ## Ingredients ##
*  1 medium tomato, chopped 
*  1 medium potato, diced 
*  8 marinated beef sprouts 
*  2 tablespoons butter 
*  1 teaspoon dried thyme 
*  1 stick artichoke hearts, minced 
 ## Instructions ##
1)  Use very finely chopped tomatoes and celery with low cut off sauce. 
1)  Set aside. 
1)  When I'm paaing, squeeze hot grease with hand. When my machine starts to start to turn high. 
1)  Now, tactfully but slightly the sun, steam the tomatoes until they start to turn golden brown (about 5 minutes). Watch them carefully. 
1)  If your sun heats, place them on tray at the stove for 15-20 minutes. 
1)  When they smell like burning, lower heat again. 
1)  Once oil is fully shaking the tomatoes you added them in 1 tablespoon. 
1)  Serve with poppy seeds; they will have a nice coating in the bottom. povertyid knifePicked up and bring back times aroun

In [ ]:
# gold standard
#df_eval = gold_st
model.to(args.device)
test_input = test.reset_index()
df_eval_ner = test_input["NER"]  #NER
df_eval_dir = test_input["directions"]

In [ ]:
def get_raw_input(NER):
    test_str = NER
    test_str = test_str.replace("[","")
    test_str = test_str.replace("]","")
    test_str = test_str.replace("\"","")

    return test_str

def get_instr(markdown):
  markdown = markdown.split("\n")
  if ' ## Instructions ##' in markdown:
    instr_index = markdown.index(' ## Instructions ##')
    
    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")

    if "" in output:
        output.remove("")    
    
    return output

  elif '## Instructions ##' in markdown:
    instr_index = markdown.index('## Instructions ##')

    output = [i for i in markdown[instr_index+1:]]

    if " " in output:
        output.remove(" ")
        
    if "" in output:
        output.remove("")    

    return output
    
  else:
    return ["failed to generate"]


def generate_recipe(raw_text):

    prepared_input = ' <RECIPE_START> <INPUT_START> ' + raw_text.replace(',', ' <NEXT_INPUT> ').replace(';', ' <INPUT_END>')
    context_tokens = tokenizer.encode(prepared_input)[0:-1]
    out = sample_sequence(
                model=model,
                context=context_tokens,
                tokenizer=tokenizer,
                length=800,
                device = args.device
            )
    out = out[0, len(context_tokens):].tolist()
    text = tokenizer.decode(out, clean_up_tokenization_spaces=True)
    if "<RECIPE_END>" not in text:
      # print(text)
      print("Failed to generate, recipe's too long")
      full_text = prepared_input + text
      # print(full_text)
      return generate_recipe(raw_text)
    else:
      full_text = prepared_input + text
      # print(full_text)
      markdown = re.sub("<RECIPE_(START|END)>", "", full_text)
      recipe_n_title = markdown.split("<TITLE_START>")
      if len(recipe_n_title)<=1:
        return generate_recipe(raw_text)
      title = "# " + recipe_n_title[1].replace("<TITLE_END>", "") + " #\n"
      markdown = recipe_n_title[0].replace("<INPUT_START>", "## Input ingredients ##\n`").replace("<INPUT_END>", "`\n")
      markdown = markdown.replace("<NEXT_INPUT>", "`\n`").replace("<INGR_START>", "## Ingredients ##\n* ").replace("<NEXT_INGR>", "\n* ").replace("<INGR_END>", "\n")
      #markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n\"").replace("<NEXT_INSTR>", "\n\"").replace("<INSTR_END>", "\n")
      markdown = markdown.replace("<INSTR_START>", "## Instructions ##\n").replace("<NEXT_INSTR>", "\n").replace("<INSTR_END>", "\n")
      markdown = re.sub("$ +#", "#", markdown)
      markdown = re.sub("( +`|` +)", "`", markdown)
  
      return markdown

In [ ]:
df_eval_dir

0     ["Preheat oven to 350 degrees. Grease muffin t...
1     ["Preheat oven to 350F.", "Microwave chocolate...
2     ["Sprinkle pepper on both sides of veal and ro...
3     ["In a small bowl, combine the first six ingre...
4     ["Brown and drain meat and onions.", "Add all ...
                            ...                        
95    ["Place raw vegetables in food processor. Proc...
96    ["add to food processor and turn on.", "blend ...
97    ["Heat oven to 350\u00b0.", "Grease and flour ...
98    ["Combine and pour over beets.", "Let set over...
99    ["Butter an 8x8-inch or 9x9-inch square pan.",...
Name: directions, Length: 100, dtype: object

In [ ]:
torch.manual_seed(0)
# generate recipes derived from gold standard ones
test_size = 100
replicated_size = 10
directions =  [[] for i in range(test_size)]

for i in range(test_size):
  raw_text = get_raw_input(df_eval_ner[i])
  print("\n Generating the recipe: ",i)

  for j in range(replicated_size):
    print("recipe: ",j)
    md = generate_recipe(raw_text)
    directions[i].append(get_instr(md))


 Generating the recipe:  0
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  1
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  2
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  3
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  4
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  5
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  6
recipe:  0
recipe:  1
recipe:  2
recipe:  3
recipe:  4
recipe:  5
recipe:  6
recipe:  7
recipe:  8
recipe:  9

 Generating the recipe:  7
recipe

In [ ]:
#func of cos_sim
import re
import math
from collections import Counter


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    word = re.compile(r'\w+')
    words = word.findall(text)
    return Counter(words)


def get_result(content_a, content_b):
    text1 = content_a
    text2 = content_b

    vector1 = text_to_vector(text1)
    vector2 = text_to_vector(text2)

    cosine_result = get_cosine(vector1, vector2)
    return cosine_result

In [ ]:
#cosine_similarity

def COS_SIM(df_eval_dir,directions):
  """
  df_eval_dir := gold_standard recipes
  directions  := corresponding recipes """

  avg = 0

  for i in range(len(directions)):
    best = 0
    for j in range(len(directions[i])):
      cos = get_result(" ".join(eval(df_eval_dir[i])),
                      " ".join(directions[i][j]))
      best = max(best, cos)

    avg += best

  avg = avg/len(directions)

  print("avg:", avg)

In [ ]:
COS_SIM(df_eval_dir,directions)

avg: 0.5582292684602654


In [ ]:
!pip install jiwer==2.2.0
!pip install -U nltk

Requirement already up-to-date: nltk in /usr/local/lib/python3.7/dist-packages (3.6.2)


In [ ]:
import nltk
import nltk.translate.bleu_score as bleu
from nltk.translate.bleu_score import SmoothingFunction

import nltk.translate.gleu_score as gleu
import nltk.translate.meteor_score as meteor

In [ ]:
# Helper Func.
def list_to_words(recipe):
  words = []
  for i in recipe:
    words += i.split()

  return words

# New BLEU/GLEU
def bleu_score(recipe, refer):
    hyp = list_to_words(eval(recipe))
    refs = []
    for i in refer:
        # print(list_to_words(i))
        refs.append(list_to_words(i))

    smoothie = SmoothingFunction().method5
    score_ref_a = bleu.sentence_bleu(refs, hyp, smoothing_function=smoothie, weights=(1,0,0,0))
    return score_ref_a

def gleu_score(recipe, refer):
    hyp = list_to_words(eval(recipe))
    refs = []
    for i in refer:
        refs.append(list_to_words(i))

    score_ref_a = gleu.sentence_gleu(refs, hyp)
    return score_ref_a

In [ ]:
bleu_avg = []
gleu_avg = []

for i in range(len(directions)):
  # print(bleu_score(df_eval_dir[i], directions[i]))
  bleu_avg.append(bleu_score(df_eval_dir[i], directions[i]))

for i in range(len(directions)):
  gleu_avg.append(gleu_score(df_eval_dir[i], directions[i]))
print(np.mean(bleu_avg))
print(np.mean(gleu_avg))

0.7910421396331373
0.09752564271635114
